<a href="https://colab.research.google.com/github/htried/wikimedia-llm-finetune/blob/main/mpt_7b_instruct_finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To prevent Colab from timing out, open console in the browser and paste/run the following:

```
interval = setInterval(function() {
    console.log("working")
    var selector = "#top-toolbar > colab-connect-button"
    document.querySelector(selector).shadowRoot.querySelector("#connect").click()
    setTimeout(function() {
            document.querySelector(selector).shadowRoot.querySelector("#connect").click()
    }, 1000)
}, 60*1000)
```

# Install

In [1]:
!pip install --upgrade transformers einops datasets accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 51.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.1/519.1 kB 53.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 33.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 34.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 120.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 86.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.9 MB/s eta 0:00:00


# Import

In [2]:
import torch
import transformers
from transformers import TrainingArguments, Trainer
from transformers import AutoModelForSequenceClassification, AdamW
from transformers import StoppingCriteria, StoppingCriteriaList, TextIteratorStreamer
from datasets import load_dataset
from torch.utils.data import DataLoader, Dataset
from typing import Any, Dict, Tuple, Union
import warnings
from threading import Event, Thread
import textwrap
from datasets import concatenate_datasets


import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# Set up

In [3]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) Y
Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' cr

In [4]:
prompt_template = {
    "description": "A shorter template to experiment with.",
    "prompt_input": "Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\n{instruction}\n\n### Input:\n{input}\n\n### Response:\n",
    "response_split": "### Response:"
}

class Prompter(object):
    def __init__(self, verbose: bool = False):
        self._verbose = verbose
        self.template = prompt_template
        if self._verbose:
            print(
                f"Using prompt template alpaca: {self.template['description']}"
            )

    def generate_prompt(
        self,
        instruction: str,
        input: str,
        output: str,
    ) -> str:
        # returns the full prompt from instruction and optional input
        # if a label (=response, =output) is provided, it's also appended.
        res = self.template["prompt_input"].format(
            instruction=instruction, input=input
        )

        if self._verbose:
            print(res)
        return f"{res}{output}"

    def get_response(self, output: str) -> str:
        return output.split(self.template["response_split"])[1].strip()

In [5]:
def train(
    base_model='mosaicml/mpt-7b-instruct',
    cutoff_len=256,
    data_path='htriedman/wiki-sparql',
    train_data_size=100_000,
    val_data_size=10_000,
    num_epochs=3,
    learning_rate=3e-4,
  ):
  prompter = Prompter()
  device_map = "auto"

  model = transformers.AutoModelForCausalLM.from_pretrained(
    base_model,
    trust_remote_code=True,
    torch_dtype=torch.bfloat16,
    device_map=device_map,
  )
  tokenizer = transformers.AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
  tokenizer.add_tokens(['{', '}', '[', ']', '\n', '\t' '(', ')'])

  if tokenizer.pad_token_id is None:
      warnings.warn(
          "pad_token_id is not set for the tokenizer. Using eos_token_id as pad_token_id."
      )
      tokenizer.pad_token = tokenizer.eos_token
  tokenizer.padding_side = "right" # "left"

  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  model.eval()
  model.to(device=device, dtype=torch.bfloat16)

  for name, param in model.named_parameters():
    if not ("31" in name or "30" in name or "29" in name or "28" in name or "27" in name or "26" in name or "25" in name or "24" in name or "23" in name or "22" in name):
        param.requires_grad = False


  def tokenize(prompt, add_eos_token=True):
      result = tokenizer(
          prompt,
          truncation=True,
          max_length=cutoff_len,
          padding=False,
          return_tensors=None,
      )
      if (
          result["input_ids"][-1] != tokenizer.eos_token_id
          and len(result["input_ids"]) < cutoff_len
          and add_eos_token
      ):
          result["input_ids"].append(tokenizer.eos_token_id)
          result["attention_mask"].append(1)

      result["labels"] = result["input_ids"].copy()

      return result

  def generate_and_tokenize_prompt(data_point):
      full_prompt = prompter.generate_prompt(
          data_point["instruction"],
          data_point["input"],
          data_point["output"],
      )
      return tokenize(full_prompt)

  data = load_dataset(data_path)
  train_data = (
      data['train'].shuffle()
      .select(range(train_data_size))
      .map(generate_and_tokenize_prompt)
  )

  val_data = (
      data['test'].shuffle()
      .select(range(val_data_size))
      .map(generate_and_tokenize_prompt)
  )

  trainer = transformers.Trainer(
      model=model,
      train_dataset=train_data,
      eval_dataset=val_data,
      args=transformers.TrainingArguments(
          output_dir='./results',
          per_device_train_batch_size=16,
          per_device_eval_batch_size=64,
          warmup_steps=100,
          num_train_epochs=num_epochs,
          learning_rate=learning_rate,
          # fp16=True,
          logging_steps=500,
          # optim="adamw_torch",
          evaluation_strategy="steps",
          save_strategy="steps",
          eval_steps=1500,
          save_steps=1500,
          save_total_limit=3,
          load_best_model_at_end=True,
      ),
      data_collator=transformers.DataCollatorForSeq2Seq(
          tokenizer, pad_to_multiple_of=8, return_tensors="pt", padding=True
      ),
  )
  model.config.use_cache = False
  trainer.train()

  model.save_pretrained("./results")
  model.push_to_hub('htriedman/mpt-7b-instruct-wiki-sparql')

# Train!

In [ ]:
train(train_data_size=150_000, val_data_size=15_000)

A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-7b-instruct:
- configuration_mpt.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-7b-instruct:
- norm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-7b-instruct:
- flash_attn_triton.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-7b-instruct:
- attention.py
- flash_attn_triton.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-7b-instruct:
- blocks.py
- norm.py
- attention.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-7b-instruct:
- custom_embedding.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-7b-instruct:
- meta_init_context.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

Generating test split:   0%|          | 0/15103 [00:00<?, ? examples/s]

Map:   0%|          | 0/150000 [00:00<?, ? examples/s]

Map:   0%|          | 0/15000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a GPTNeoXTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss
